# Separação treino-teste

In [1]:
from loader_and_preprocess_funcs import *

In [2]:
data =load__data()

In [3]:
top15 = ['Neighborhood','House.Style','Overall.Qual','Full.Bath','Year.Remod.Add','Lot.Area','TotRms.AbvGrd','Fireplaces','Heating.QC','Lot.Frontage',
             'Foundation','Gr.Liv.Area','Kitchen.AbvGr','Open.Porch.SF','Roof.Style','SalePrice']

data = choose_feat(data,top15)

In [4]:
data

,Neighborhood,House.Style,Overall.Qual,Full.Bath,Year.Remod.Add,Lot.Area,TotRms.AbvGrd,Fireplaces,Heating.QC,Lot.Frontage,Foundation,Gr.Liv.Area,Kitchen.AbvGr,Open.Porch.SF,Roof.Style,SalePrice
0,NAmes,1Story,6,1,1960,31770,7,2,Fa,141.0,CBlock,1656,1,62,Hip,215000
1,NAmes,1Story,5,1,1961,11622,5,0,TA,80.0,CBlock,896,1,0,Gable,105000
2,NAmes,1Story,6,1,1958,14267,6,0,TA,81.0,CBlock,1329,1,36,Hip,172000
3,NAmes,1Story,7,2,1968,11160,8,2,Ex,93.0,CBlock,2110,1,0,Hip,244000
4,Gilbert,2Story,5,2,1998,13830,6,1,Gd,74.0,PConc,1629,1,34,Gable,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,Mitchel,SLvl,6,1,1984,7937,6,0,TA,37.0,CBlock,1003,1,0,Gable,142500
2926,Mitchel,1Story,5,1,1983,8885,5,0,TA,NaN,CBlock,902,1,0,Gable,131000
2927,Mitchel,SFoyer,5,1,1992,10441,6,0,TA,62.0,PConc,970,1,32,Gable,132000
2928,Mitchel,1Story,5,1,1975,10010,6,1,Gd,77.0,CBlock,1389,1,38,Gable,170000


In [5]:
data = preprocessor(data)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2273 entries, 1 to 2929
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Neighborhood       2273 non-null   object 
 1   House.Style        2273 non-null   object 
 2   Overall.Qual       2273 non-null   object 
 3   Full.Bath          2273 non-null   object 
 4   Year.Remod.Add     2273 non-null   int64  
 5   TotRms.AbvGrd      2273 non-null   object 
 6   Fireplaces         2273 non-null   object 
 7   Heating.QC         2273 non-null   object 
 8   Foundation         2273 non-null   object 
 9   Kitchen.AbvGr      2273 non-null   object 
 10  Roof.Style         2273 non-null   object 
 11  Lot.Area_log       2273 non-null   float64
 12  Lot.Frontage_log   1878 non-null   float64
 13  Gr.Liv.Area_log    2273 non-null   float64
 14  SalePrice_log      2273 non-null   float64
 15  Open.Porch.SF_bin  2273 non-null   object 
dtypes: float64(4), int64(1), obje

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
y = data['SalePrice_log']
X = data.drop(columns=['SalePrice_log'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline Global

In [9]:

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,PolynomialFeatures
from sklearn.model_selection import cross_val_score

In [67]:
num_cols =['Gr.Liv.Area_log','Lot.Area_log','Lot.Frontage_log','Year.Remod.Add']
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=3, include_bias=False))
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot',  OneHotEncoder(handle_unknown='ignore', sparse_output=False,drop='first')),
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline,      num_cols),
    ('cat', categorical_pipeline,  cat_cols),
],
remainder='passthrough')



# Modelos

ElasticNet, SVM, Trees e ensamble(cpa)

In [11]:
def print_cv_results(model_name, cv_results):
    cv_results = -cv_results

    print(f'{model_name} CV results:')
    print(cv_results)

    mean_rmse = cv_results.mean()
    std_rmse = cv_results.std()

    print(f'RMSE: {mean_rmse:.2f} +/- {std_rmse:.2f}')
    print()

In [54]:
result = pd.DataFrame()

# Dummy model

In [12]:
from sklearn.dummy import DummyRegressor

dummy_reg = Pipeline([
    ('preprocessing', preprocessor),
    (
        'regression', DummyRegressor(strategy='mean'),
    ),
])

In [ ]:


cv = 5

dummy_reg_cv = cross_val_score(
    dummy_reg,
    X_train,
    y_train,
    cv=cv,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
)
print_cv_results('Dummy Regressor', dummy_reg_cv)


Dummy Regressor CV results:
[0.15596792 0.15681443 0.16070465 0.15443107 0.15120019]
RMSE: 0.16 +/- 0.00



In [58]:

result['Dummy'] = dummy_reg_cv

Rmse de 0.16(escala log)

# Linear Model 

In [14]:
from sklearn.linear_model import LinearRegression

lin_reg = Pipeline([
    ('preprocessing', preprocessor),
    ('regression', LinearRegression()),
])

In [15]:
cv = 5

lin_reg_cv = cross_val_score(
    lin_reg,
    X_train,
    y_train,
    cv=cv,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
)
print_cv_results('Linear Regressor', lin_reg_cv)

Linear Regressor CV results:
[0.05465166 0.0547433  0.05556746 0.05174398 0.05500285]
RMSE: 0.05 +/- 0.00



In [59]:
result['Linear Reg'] = lin_reg_cv

Rmse de 0.05 (em escala log)

# Elastic Net Regressor

In [16]:
from sklearn.linear_model import ElasticNet 
elastic_net = Pipeline([
    ('preprocessing', preprocessor),
    ('elasticnet', ElasticNet()),
])

In [17]:
import optuna

/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [77]:
def objective(trial):
    alpha         = trial.suggest_float('alpha',      1e-3, 100.0, log=True)
    l1_ratio      = trial.suggest_float('l1_ratio',   0.1,   1.0)
    fit_intercept = trial.suggest_categorical('fit_intercept', [False, True])
    max_iter      = trial.suggest_int('max_iter',     1000,  5000)

    use_scaler    = trial.suggest_categorical('use_scaler', [False, True])
    poly_degree   = trial.suggest_int('poly_degree', 1,  5)

    num_steps = [
        ('imputer', SimpleImputer(strategy='mean'))
    ]
    if use_scaler:
        scaler_option = trial.suggest_categorical('scaler_option', ['mean', 'std'])
    poly_degree   = trial.suggest_int('poly_degree',   1,     5)

    num_steps = [('imputer', SimpleImputer(strategy='mean'))]
    if use_scaler:
        if scaler_option == 'mean':
            num_steps.append(('scaler',
                              StandardScaler(with_mean=True,  with_std=False)))
        else:
            num_steps.append(('scaler',
                              StandardScaler(with_mean=False, with_std=True)))
    
    num_steps.append(('poly',
                      PolynomialFeatures(degree=poly_degree,
                                         include_bias=False)))
            
    numeric_pipeline = Pipeline(num_steps)

    categorical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot',  OneHotEncoder(handle_unknown='ignore',
                                  sparse_output=False,
                                  drop='first')),
    ])

    preprocessor = ColumnTransformer(
        [
            ('num', numeric_pipeline,      num_cols),
            ('cat', categorical_pipeline,  cat_cols),
        ],
        remainder='passthrough'
    )

    model = Pipeline([
        ('preproc', preprocessor),
        ('est', ElasticNet(
            alpha=alpha,
            l1_ratio=l1_ratio,
            fit_intercept=fit_intercept,
            max_iter=max_iter,
            random_state=42
        )),
    ])

    neg_mse = cross_val_score(
        model, X_train, y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    ).mean()

    return neg_mse



In [78]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=42)) 

[I 2025-05-03 15:39:23,702] A new study created in memory with name: no-name-8d073190-14b8-4469-9089-e5f3c30e4ebb


In [79]:
study.optimize(objective, n_trials=100)

/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.947e+00, tolerance: 4.015e+00
  model = cd_fast.enet_coordinate_descent(
/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.253e+00, tolerance: 4.016e+00
  model = cd_fast.enet_coordinate_descent(
/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

In [80]:
optuna.visualization.plot_optimization_history(study)

In [81]:
best_params = study.best_params 
print(f"Best Hyperparameters: {best_params}")

Best Hyperparameters: {'alpha': 0.0012599577693434842, 'l1_ratio': 0.3898712490248605, 'fit_intercept': True, 'max_iter': 1479, 'use_scaler': False, 'poly_degree': 4}


In [83]:
from optuna.visualization import *
import plotly
import optuna
import nbformat 

In [84]:
optuna.visualization.plot_slice(study, params=['alpha', 'l1_ratio', 'fit_intercept', 'max_iter','use_scaler','poly_degree','scaler_option'])

In [85]:
optuna.visualization.plot_param_importances(study)

In [86]:
best_alpha = best_params['alpha']
best_l1_ratio = best_params['l1_ratio']
best_fit_intercept = best_params['fit_intercept']
best_max_iter = best_params['max_iter']




In [87]:
numeric_pipeline_elastic = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=4, include_bias=False))
])

categorical_pipeline_elastic = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot',  OneHotEncoder(handle_unknown='ignore', sparse_output=False,drop='first')),
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline_elastic,      num_cols),
    ('cat', categorical_pipeline_elastic,  cat_cols),
],
remainder='passthrough')

elastic = Pipeline([
    ('prepoc',preprocessor),
        ('est', ElasticNet(
            alpha=best_alpha,
            l1_ratio=best_l1_ratio,
            fit_intercept=best_fit_intercept,
            max_iter=best_max_iter,
            random_state=42
        )),
    
])

In [88]:
est_cv = cross_val_score(
        elastic, X_train, y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
print_cv_results('Elastic Net Regressor',est_cv)

/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.841e+00, tolerance: 3.519e-03
  model = cd_fast.enet_coordinate_descent(
/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.841e+00, tolerance: 3.529e-03
  model = cd_fast.enet_coordinate_descent(
/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

Elastic Net Regressor CV results:
[0.00337859 0.0033661  0.00349701 0.00287815 0.00304782]
RMSE: 0.00 +/- 0.00



/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.905e+00, tolerance: 3.584e-03
  model = cd_fast.enet_coordinate_descent(


In [89]:
result['ElsticNet'] = est_cv

# Random Forest Regressor

In [90]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def objective(trial): 
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 10, 50) 
    min_samples_split = trial.suggest_int('min_samples_split', 2, 32) 
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 32) 

    num_steps = [
        ('imputer', SimpleImputer(strategy='mean'))
    ]
    poly_degree   = trial.suggest_int('poly_degree',1,3)

    num_steps = [('imputer', SimpleImputer(strategy='mean'))]
    
    num_steps.append(('poly',
                      PolynomialFeatures(degree=poly_degree,
                                         include_bias=False)))
            
    numeric_pipeline = Pipeline(num_steps)

    categorical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot',  OneHotEncoder(handle_unknown='ignore',
                                  sparse_output=False,
                                  drop='first')),
    ])

    preprocessor = ColumnTransformer(
        [
            ('num', numeric_pipeline,      num_cols),
            ('cat', categorical_pipeline,  cat_cols),
        ],
        remainder='passthrough'
    )
    model = Pipeline([
        ('preproc', preprocessor),
        ('randomforest', RandomForestRegressor(
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        min_samples_split=min_samples_split, 
        min_samples_leaf=min_samples_leaf,
        random_state=42
        )),
    ])

    neg_mse = cross_val_score(
        model, X_train, y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    ).mean()

    return neg_mse

In [102]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=42)) 

[I 2025-05-03 15:45:31,619] A new study created in memory with name: no-name-e14df4ff-2051-4e62-8bcb-f2c630aa1740


In [103]:
study.optimize(objective, n_trials=100)

[I 2025-05-03 15:45:40,152] Trial 0 finished with value: -0.004594551395352957 and parameters: {'n_estimators': 437, 'max_depth': 48, 'min_samples_split': 24, 'min_samples_leaf': 20, 'poly_degree': 1}. Best is trial 0 with value: -0.004594551395352957.
[I 2025-05-03 15:45:53,958] Trial 1 finished with value: -0.00445005723682228 and parameters: {'n_estimators': 240, 'max_depth': 12, 'min_samples_split': 28, 'min_samples_leaf': 20, 'poly_degree': 3}. Best is trial 1 with value: -0.00445005723682228.
[I 2025-05-03 15:45:55,329] Trial 2 finished with value: -0.004058536747106786 and parameters: {'n_estimators': 118, 'max_depth': 49, 'min_samples_split': 27, 'min_samples_leaf': 7, 'poly_degree': 1}. Best is trial 2 with value: -0.004058536747106786.
[I 2025-05-03 15:45:58,873] Trial 3 finished with value: -0.0043284876421504324 and parameters: {'n_estimators': 265, 'max_depth': 22, 'min_samples_split': 18, 'min_samples_leaf': 14, 'poly_degree': 1}. Best is trial 2 with value: -0.0040585367

In [104]:
optuna.visualization.plot_optimization_history(study)

In [105]:
best_params = study.best_params 
print(f"Best Hyperparameters: {best_params}")

Best Hyperparameters: {'n_estimators': 930, 'max_depth': 13, 'min_samples_split': 8, 'min_samples_leaf': 2, 'poly_degree': 1}


In [106]:
optuna.visualization.plot_slice(study, params=['n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf','poly_degree'])

In [107]:
optuna.visualization.plot_param_importances(study)

In [108]:
best_n= best_params['n_estimators']
best_max_depth = best_params['max_depth']
best_min_samples_split = best_params['min_samples_split']
best_min_samples_leaf = best_params['min_samples_leaf']

In [111]:
numeric_pipeline_random_tree = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=1, include_bias=False))
])

categorical_pipeline_elastic = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot',  OneHotEncoder(handle_unknown='ignore', sparse_output=False,drop='first')),
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline_elastic,      num_cols),
    ('cat', categorical_pipeline_elastic,  cat_cols),
],
remainder='passthrough')

rforest = Pipeline([
    ('prepoc',preprocessor),
    ('randomforest', RandomForestRegressor(
        n_estimators=best_n, 
        max_depth=best_max_depth, 
        min_samples_split=best_min_samples_split, 
        min_samples_leaf=best_min_samples_leaf,
        random_state=42
        )),
    
])

In [112]:
rforest_cv = cross_val_score(
        rforest, X_train, y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )

In [113]:
print_cv_results('Random Forest Regressor',rforest_cv)

Random Forest Regressor CV results:
[0.004101   0.0039174  0.003651   0.00360113 0.00359323]
RMSE: 0.00 +/- 0.00



In [114]:
result['RandomForest'] = rforest_cv

# Avaliando os modelos

In [115]:
result

,Dummy,Linear Reg,ElsticNet,RandomForest
0,-0.155968,-0.054652,-0.003379,-0.004101
1,-0.156814,-0.054743,-0.003366,-0.003917
2,-0.160705,-0.055567,-0.003497,-0.003651
3,-0.154431,-0.051744,-0.002878,-0.003601
4,-0.151200,-0.055003,-0.003048,-0.003593


In [122]:
print('Média de NRMSE do Dummy Model', result['Dummy'].mean())
print('Média de NRMSE do Linear model', result['Linear Reg'].mean())
print('Média de NRMSE do ElasticNet model', result['ElsticNet'].mean())
print('Média de NNRMSE do RandomForest model', result['RandomForest'].mean())

Média de NRMSE do Dummy Model -0.1558236528631532
Média de NRMSE do Linear model -0.05434184924197355
Média de NRMSE do ElasticNet model -0.0032335357104508666
Média de NNRMSE do RandomForest model -0.003772751297930811


Os modelo ElasticNet e RandomForest se destacam do resto quando se compara o desempenho em média dos dois, vamos ver em porcentagem para ter uma noção melhor, e depois vamos comparar sua variância também

In [138]:
mean_elastic = 100*(10**np.sqrt(-result['ElsticNet'].mean())-1)
mean_rforest = 100*(10**np.sqrt(-result['RandomForest'].mean())-1) 
print(f'Média em porcentagem do RMSE do ElasticNet model {mean_elastic:.2f}%')
print(f'Média em porcentagem do RMSE do RandomForest model {mean_rforest:.2f}%')

Média em porcentagem do RMSE do ElasticNet model 13.99%
Média em porcentagem do RMSE do RandomForest model 15.19%


In [123]:
print('Variância de NRMSE do Dummy Model',      result['Dummy'].var())
print('Variância de NRMSE do Linear model',     result['Linear Reg'].var())
print('Variância de NRMSE do ElasticNet model', result['ElsticNet'].var())
print('Variância de NRMSE do RandomForest model', result['RandomForest'].var())

Variância de NRMSE do Dummy Model 1.2035556115781293e-05
Variância de NRMSE do Linear model 2.2362753326933676e-06
Variância de NRMSE do ElasticNet model 6.720621290309157e-08
Variância de NRMSE do RandomForest model 5.129295387526053e-08


Podemos ver que o RandomForest tem uma vairância menor que o ElasticNet, mas não parece ser algo muito gritante(mas estamos analisando esses resultados na escala log, vamos analisar em porcentagem)

In [126]:
print('Variancia em porcentagem do RMSE do ElasticNet model',100*(10**np.sqrt(result['ElsticNet'].var()) - 1))
print('Variancia em porcentagem do RMSE do RandomForest model',100*(10**np.sqrt(result['RandomForest'].var()) - 1))

Variancia em porcentagem do RMSE do ElasticNet model 0.059710406480339984
Variancia em porcentagem do RMSE do RandomForest model 0.05216242681529426


As duas variância estão em torno de 5%/6%. Para termos uma noção estatísitca melhor vamos analisar o coeficiente de variação desses modelos.

In [131]:

pct_errors = (10**result['RandomForest'] - 1) * 100

cv = pct_errors.std() / pct_errors.mean() * 100

print(f'CV do modelo RandomForest: {cv:.2f}%')

CV do modelo RandomForest: -5.98%


In [130]:
pct_errors = (10**result['ElsticNet'] - 1) * 100

cv = pct_errors.std() / pct_errors.mean() * 100

print(f'CV do modelo ElasticNet: {cv:.2f}%')

CV do modelo ElasticNet: -7.99%


Partindo agora para o t-test:

In [145]:
from scipy.stats import ttest_rel

In [ ]:


scores_dict = result.to_dict(orient='list')
means = {name: np.mean(vals) for name, vals in scores_dict.items()}
best_name = max(means, key=means.get)
print(f"Melhor modelo: {best_name} (erro médio = {means[best_name]:.4f})\n")

alpha = 0.05
n_models = len(scores_dict) - 1
alpha_corr = alpha / n_models
for name, scores in scores_dict.items():
    if name == best_name:
        continue

    best_scores = scores_dict[best_name]
    other_scores = scores
    stat, p = ttest_rel(best_scores, other_scores)

    delta = means[name] - means[best_name]

    signif = 'SIM' if p < alpha_corr else 'NÃO'
    print(
        f"{name:15s} → Δmédia = {delta:+.4f} | "
        f"p-valor = {p:.4f} | Significativo p < {alpha_corr:.4f}? {signif}"
    )


Melhor modelo: ElsticNet (erro médio = -0.0032)

0.016666666666666666
Dummy           → Δmédia = -0.1526 | p-valor = 0.0000 | Significativo p < 0.0167? SIM
Linear Reg      → Δmédia = -0.0511 | p-valor = 0.0000 | Significativo p < 0.0167? SIM
RandomForest    → Δmédia = -0.0005 | p-valor = 0.0066 | Significativo p < 0.0167? SIM


Aplicamos testes pareados (t-test) comparando fold a fold o erro médio do ElasticNet contra Dummy, Linear Reg e RandomForest, usando correção de Bonferroni (α_corr=0,0167). Em todas as comparações obtivemos p-valores inferiores a α_corr, portanto podemos  rejeitar a hipótese nula de que não existia diferença da média de erro do ElasticNet para os outros modelos. Concluímos que os outros modelos apresentam erro médio estatisticamente maior que o ElasticNet. Entretanto, a magnitude da diferença para o RandomForest (Δ≈0,0005) é muito pequena e possivelmente irrelevante na prática.